In [1]:
# from pathlib import Path
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import time

In [17]:
# linear model
from sklearn.linear_model import ElasticNet, Lasso, Ridge, BayesianRidge, HuberRegressor, RANSACRegressor

# boosting model
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# other models
from sklearn.svm import SVR, LinearSVR, NuSVR            
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

In [18]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV ,cross_val_score ,cross_validate, RandomizedSearchCV
# from mlxtend.classifier import EnsembleVoteClassifier
# from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import VotingRegressor


from scipy.stats import spearmanr
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
# from yellowbrick.model_selection import ValidationCurve, LearningCurve

In [4]:
df_all = pd.read_pickle('data/merged data7.pkl.zip')
df_all.reset_index(drop=True, inplace=True)

In [5]:
df_all['Ticker_code'] = df_all['Ticker'].astype('category').cat.codes
df_all['date_period'] = (df_all['date'] - min(df_all['date'])).dt.days + 1
df_all.sort_values('date', inplace=True)

In [6]:
df_data = df_all[list(df_all.columns[3:])]
df_target = df_all['alpha']

In [10]:
timecv= TimeSeriesSplit(n_splits=3)

In [ ]:
#SOFTVOTING
voting_regressor = VotingRegressor(
    estimators=[
        ('xgb', XGBRegressor()),
        ('lgbm', LGBMRegressor()),
        ('cat', CatBoostRegressor(verbose=0))
    ],
    weights=[1, 1, 1]
)
MSE_voting = cross_val_score(voting_regressor, df_data, df_target, cv=timecv, scoring='neg_mean_squared_error').mean()
print("Soft Voting Regressor : ", MSE_voting)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.054431 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 21534
[LightGBM] [Info] Number of data points in the train set: 74222, number of used features: 89
[LightGBM] [Info] Start training from score -0.799139
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.087821 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 21613
[LightGBM] [Info] Number of data points in the train set: 148444, number of used features: 89
[LightGBM] [Info] Start training from score -0.314705
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.250394 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 21688
[LightGBM] [Info] Number of data points in the train set: 222666, number of used features: 89
[LightGBM] [Info] St

In [19]:
random_search = RandomizedSearchCV(
    VotingRegressor(
        estimators=[
            ('xgb', XGBRegressor()),
            ('lgbm', LGBMRegressor()),
            ('cat', CatBoostRegressor(verbose=0))
        ]
    ),
    param_distributions=param_dist,
    n_iter=20,  
    cv=timecv,  
    scoring='neg_mean_squared_error',
    random_state=42,
    verbose=1
)

random_search.fit(df_data, df_target)
print("Best weights:", random_search.best_params_)
print("Best score:", random_search.best_score_)

NameError: name 'param_dist' is not defined